## Import Library

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Make Request

In [2]:
base_url = "https://homes.trovit.my/index.php/cod.search_adwords_homes/ppc_landing_type.2/type.1/what_d.house/sug.0/tracking./isUserSearch.1/origin.11/order_by.relevance/city.Johor%20Bahru/"

# To get the html contents
r = requests.get(base_url)
c = r.text

## Test Communication

In [3]:
print(r)

<Response [200]>


## Parse HTML

In [4]:
# To parse the html
soup = BeautifulSoup(c,"html.parser")

## Extract First and Last Page

In [5]:
# To extract the first and last page numbers
page = soup.find("p",id = "paginate").find_all("a")
start_page = page[1].text
last_page  = page[len(page)-2].text
print("Last page : ", last_page)

Last page :  5


## Calculate Number of item in Container

In [6]:
containers = soup.find("div",class_="serp-list js-serp-list").find_all("div",class_="item my js-item js-backToTrovit")
print("Number of items in the container :", len(containers))

Number of items in the container : 25


In [7]:
container = containers[0]
print(container)

<div class="item my js-item js-backToTrovit" data-id="iJjy1G1C8F" data-is-premium="false" itemprop="offers" itemscope="" itemtype="http://schema.org/SingleFamilyResidence https://schema.org/Offer"> <div class="picture"> <div class="image"> <img alt="Photo Winter Sonata 3 Storey Cluster House" class="scaleWidth " height="120" itemprop="image" src="//img-ap-1.trovit.com/img1my/iJjy1G1C8F/iJjy1G1C8F.1_10.jpg" width="160"/> <span class="new_building">newly built</span> </div> <div class="source_and_date"> <div class="first_letter">i</div> <div class="source_info"> <small class="source"><span>iProperty</span></small> <small class="date">30+ days ago</small> </div>
</div></div> <div class="details"> <h4 itemprop="description"> <a class="js-item-title" href="http://rd.trovit.com/id.iJjy1G1C8F/origin.11/section.97/section_type.1/country.my/vertical.homes/iuid.9477web-modphp-045c88881f9c767/tracking/what_d.house/type.1/page.1/cookieid.115135211985c88881f9b5aa0.69764144/pos.1/" rel="nofollow" ta

## Extract Data

In [8]:
container = containers[0]
#print(block)

var_1 = container.find("div",class_="details").find("h4").text.strip()                    #Title
var_2 = container.find("div",class_="details").find("h5").text.strip()                    #Location
var_3 = container.find("div",class_="features").find("div",class_="price").text.strip()   #Price
var_4 = container.find("div",class_="features").find_all("div",class_="property")[0].text.strip()    #Bedroom
var_5 = container.find("div",class_="features").find_all("div",class_="property")[1].text.strip()    #Bathroom
var_6 = container.find("div",class_="features").find_all("div",class_="property")[2].text    #FloorSize
print(var_1)
print(var_2)
print(var_3)
print(var_4)
print(var_5)
print(var_6)

Winter Sonata 3 Storey Cluster House
Iskandar Puteri, Johor
MYR 950,000
5 bed
6 br
3461 sq. feet


## Convert to Dataframe

In [9]:
content_list = []

#for page_number in range(int(start_page),int(last_page) + 1):
for page_number in range(int(start_page),5):  #limit to until 5th page
    
    # To form the url based on page numbers
    url = base_url+str(page_number)+"/.html"
    r = requests.get(base_url+str(page_number)+"/")
    c = r.text
    soup = BeautifulSoup(c,"html.parser")
    
    containers = soup.find("div",class_="serp-list js-serp-list").find_all("div",class_="item my js-item js-backToTrovit")
    
    for container in containers:
      content_dict = {}
      content_dict["var_1"] = container.find("div",class_="details").find("h4").text.strip() 
      content_dict["var_2"] = container.find("div",class_="details").find("h5").text.strip()     
      content_dict["var_3"] = container.find("div",class_="features").find("div",class_="price").text.strip()  
      
      x1 = container.find("div",class_="features").find("span",itemprop="numberOfRooms")
      content_dict["var_4"] = x1.get_text() if x1 else ''
      
      #x2 = container.find("div",class_="features").find_all("div",class_="property")[0]
      #content_dict["var_5"] = x2.get_text() if x2 else ''
      
      x3 = container.find_all("div",class_="property")[-1]
      content_dict["var_6"] = x3.get_text() if x3 else ''
      
      # To store the dictionary to into a list
      content_list.append(content_dict)
    
# To make a dataframe with the list
df = pd.DataFrame(content_list)
#print('At least {} pages scrapped, containing {} properties.' .format(last_page, len(var_1)))

In [10]:
df

,var_1,var_2,var_3,var_4,var_6
0,Winter Sonata 3 Storey Cluster House,"Iskandar Puteri, Johor","MYR 950,000",5,3461 sq. feet
1,The Mercato @ Setia Indah,"Tebrau, Johor Bahru, Johor","MYR 748,585",,3080 sq. feet
2,Cluster Factory @ Setia Business Park 2,"Tebrau, Johor Bahru, Johor","MYR 3,376,000",,7488 sq. feet
3,Meridin Walk,"Iskandar Puteri, Johor","MYR 486,000",,484 sq. feet
4,Aspira parkhomes,"Iskandar Puteri, Johor","MYR 526,000",4,3 br
5,Meridin Suites,"Iskandar Puteri, Johor","MYR 332,000",,318 sq. feet
6,Sunway Grid Shoppe & Office,"Johor Bahru, Johor","MYR 738,000",,743 sq. feet
7,Sunway Citrine Lakehomes - 2 Storey Link Cluster,"Iskandar Puteri, Johor","MYR 870,000 MYR 895,500",4,2238 sq. feet
8,Sunway Citrine Lakehomes - 3 Storey Townhouse,"Iskandar Puteri, Johor","MYR 900,000",4,2148 sq. feet
9,Sunway Citrine Lakehomes - Double Storey Terrace,"Iskandar Puteri, Johor","MYR 840,000 MYR 845,000",4,2066 sq. feet
